## Welcome to your first studio notebook! 
### This is a simple notebook to infer the parameters of a straight line model. 

It accompanies Chapter 1 of the book.
We'll continue building on it as part of your first homework assignment.

Copyright: Viviana Acquaviva (2023)

License: [BSD-3-clause](https://opensource.org/license/bsd-3-clause/)

Modified by Julieta Gruszko (2025)

#### List the names your group members below:

In [ ]:
# First, let's import the packages we'll use
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# Here are some settings to make the plots look nicer
font = {'size'   : 14}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
#matplotlib.rcParams.update({'figure.autolayout': False})
#matplotlib.rcParams['figure.dpi'] = 300

In this notebook, we generate some data that follow a linear relationship with some noise, and we use a grid search to find the best fit model, with and without considering the uncertainties. Then, we'll try some machine learning models on the same data.

These data could represent any measurements where a linear relationship is a good model for the underlying physics. Let's imagine them as measurements taken of distance traveled at difference points in time by a cart on a track moving at constant velocity.

In [ ]:
np.random.seed(16) #why are we fixing the seed?

x = np.arange(10) # returns a numpy array containing integers 0 to 9

y = 2*x + 5 + np.random.randn(10) #generate some data with random gaussian scatter and add it to 2*x+5. 

### Two questions:
#### 1. What is the mean and standard deviation ($\sigma$) of the Gaussian noise being added to the y values? (Hint: VS Code should give you information about a function when you hover your mouse over it. What is randn(10) doing?)
#### 2. If the x values are times in s and the y values are distances in m, what is the "true" velocity of the cart?

Put your answers in the cell below:

In [ ]:
plt.figure(figsize=(8,6)) #
plt.scatter(x,y, c = 'red')
plt.xlabel('time (s)')
plt.ylabel('distance (m)');

In [ ]:
y = np.round(y,1) #match the book
print(y)

Let's generate a grid of slopes and intercepts, after deciding a reasonable range by looking at the graph. 

In [ ]:
slopes = np.linspace(1,3,101) # asks numpy to give us an array of 101 values from 1 to 3. Choosing 101 gives us nicer-looking numbers

intercepts = np.linspace(4,6,101) 

In [ ]:
slopes #just checking

In [ ]:
intercepts

#### Note: these are already > 10000 models (curse of dimensionality!)

For convenience, we can define two functions. The first implements our model (a straight line) and the second one is the sum of squared errors that we can use to evaluate each slope, intercept pair (ignoring uncertainties for now):
$$ \Sigma_i \Delta y_i^2 = \Sigma_i (y_i - \bar{y_i})^2 = \Sigma_i (y_i - (m x_i+b))^2 $$

where the index $i$ is over the data points, $y_i$ is the distance data, and $\bar{y_i}$ is the modeled distance at time $x_i$.

In [ ]:
def model(x,m,b):
    return m*x + b

def se(x,m,b,yobs):
    return np.sum((yobs - model(x,m,b))**2)

We can calculate the squared error for each value and save it in the "square_errs" array.

In [ ]:
square_errs = np.zeros((101,101)) #Initialization

For loops are less "pythonic" (and usually less efficient), but often easier to read and understand. Sometimes a nice way to work is to start by writing the loop version, and then going back and replacing those lines with more-efficient versions once you have your basic algorithm in place. 

In [ ]:
for i, m in enumerate(slopes):
    for j,b in enumerate(intercepts):
        square_errs[i,j] = se(x,m,b,y)

We can also do this with a one-liner by using list comprehension:

In [ ]:
square_errs = np.array([[se(x,m,b,y) for b in intercepts] for m in slopes]) 

In [ ]:
square_errs.shape #check that the array has been built properly

In [ ]:
square_errs

Note that this generates an array where the first index refers to slope and the second index refers to intercept.


#### Let's figure out which model is the best fit (lowest Squared Error).

In [ ]:
np.argmin(square_errs) #index of min; however this corresponds to flattened array

In [ ]:
indices = np.unravel_index(square_errs.argmin(), square_errs.shape) #indices of minimum value as a (row, col) pair

print(indices)

We can now derive the slope and intercept for the best-fitting model.

In [ ]:
bestm, bestb = slopes[indices[0]],intercepts[indices[1]]

In [ ]:
bestm, bestb 

### Question: How do the best-fit slope and y-intercept compare to the original ones?
When you're asked something like this, it's good to make it quantitative! An easy way to do that is to give the % disagreement.

Finally, we plot the best fit model against the data.

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x,y, c = 'red') 
plt.xlabel('time (s)')
plt.ylabel('distance (m)')
plt.plot(x, bestm * x + bestb, c = 'm'); 

#### What if there are varying distance measurement uncertainties on each data point? Let's see how our fit changes.

First let's generate some error bars for the points.

In [ ]:
np.random.seed(10)

dy = np.random.randn(10)*np.sqrt(2) #these are the uncertainties; sign doesn't matter.

In [ ]:
dy

We want to consider the uncertainties by giving more weight to data points with a lower uncertainties. This can be achieved by modifying the squared error function with an inverse weight of the uncertainties. The error function becomes the chi squared:

$$\chi^2 = \Sigma_i \frac{(y_i - \bar{y_i})^2}{\epsilon_i^2}$$

where $i$ runs over all the data points, $y_i$ is the observed value, $\bar{y_i}$ is the expected value of the model at that point, and $\epsilon_i$ is the uncertainty of the data point.



#### Your turn:
Write a function that returns the $\chi^2$ value for an array of observed points with associated errors, given the parameters of the linear model $m$, and $b$.

In [ ]:
def chi2(x,m,b,yobs,err):
    return #your expression here

We can now generate the array of the modified evaluation function, and find the indices of its minimum like we did above.

In [ ]:
allchi2 = np.array([[chi2(x,m,b,y,dy) for b in intercepts] for m in slopes]) 

In [ ]:
print(allchi2.argmin()) #index of min; however this corresponds to flattened array

indices = np.unravel_index(allchi2.argmin(), allchi2.shape) #indices of minimum value as a (row, col) pair

### Your turn again: as above, get the slope and y intercept of the best model and compare them to the results without uncertainties.

In [ ]:
#Derive the slope and intercept for best model

bestm_werr, bestb_werr = # your code here

In [ ]:
bestm_werr, bestb_werr

We can plot the data (with uncertainties) and the new best fit line.

In [ ]:
plt.errorbar(x,y, np.abs(dy), marker = 'o', markersize = 3, c = 'red', linestyle = ' ')
plt.xlabel('time (s)')
plt.ylabel('distance (m)')
plt.plot(x, bestb_werr + bestm_werr * x, c = 'c');

We can compare the two lines:

In [ ]:
plt.figure(figsize=(8,6))

plt.errorbar(x,y, np.abs(dy), marker = 'o', markersize = 3, c = 'red', linestyle = ' ')

plt.plot(x, bestb + bestm * x, c = 'm', label = 'Fit without uncertainties')

plt.plot(x, bestb_werr + bestm_werr * x, c = 'c', label = 'Fit with uncertainties')

plt.xlabel('time (s)')

plt.ylabel('distance (m)')

plt.legend();


#### Question: 
Does the change make sense? Explain what you're seeing (and why). 

### Next, we'll see how using machine learning to solve this problem differs from using a "traditional" inference approach. 

We'll use the same data as before, and try two models that we'll study in depth in the coming weeks: a Decision Tree Regressor and Linear Regression. To use them, we have to first import the models from scikit-learn. We'll also use sklearn's tools to split the data into train and test sets.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 

We'll split our data into train/test sets with a 70/30 split. In this case, that means 3 test points.

In [ ]:
np.random.seed(10) #fix for reproducibility
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=3) #create train/test split

We can take a look at what points are in each set:

In [ ]:
X_train, y_train

In [ ]:
X_test

The output of the test set is hidden: that's what we're trying to reproduce with our ML models.

We'll make a model object for each of our two models, each using the default parameters from sklearn

In [ ]:
treemodel = DecisionTreeRegressor() # default params
regmodel = LinearRegression() # default params

Let's try training our first model! scikit-learn's $\texttt{fit}$ method does the training. We need to give it our training data (both the x and y values).

In [ ]:
treemodel.fit(X_train, y_train)
regmodel.fit(X_train, y_train)

Whoops, that didn't work! Luckily the error message is quite informative, and even tells us how to fix the problem. 

#### Your turn! Fix the error and train the two models:

In [ ]:
# your code to train the decision tree model here

In [ ]:
# your code to train the linear regression model here

Notice that the print out associated with the model object returned here is quite helpful! It tells us the settings used to train the model, and you can mouse over the "i" to see that the model has been fitted. Clicking the "?" takes you to the documentation page for the model.

Now, let's use our trained models to predict the y values for the test set:

In [ ]:
y_pred_tree = treemodel.predict(X_test)
y_pred_reg = regmodel.predict(X_test)

Same error as before! By now, you should be a pro at fixing it.
### Your turn:

In [ ]:
y_pred_tree = # your code to predict tree values here
y_pred_reg = # your code to predict regression values here

Let's take a look at the results. 

In [ ]:
print(y_test, y_pred_reg, y_pred_tree) #True/predicted by LR and DT respectively

Let's calculate the mean squared error with each model.

In [ ]:
np.mean((y_test-y_pred_reg)**2)

In [ ]:
np.mean((y_test-y_pred_tree)**2)

Ultimately, our goal in building a model is often to use it to make predictions. Let's use our two linear fits and our two ML models to predict what the cart's distance will be at t = 12 s. 

I'll get you started with two of the four values:

In [ ]:
linpred = bestm * 12 + bestb
treepred = treemodel.predict(np.array(12).reshape(-1, 1))

### Your turn: 
Find the prediction of the linear fit that takes uncertainties into account and of the linear regression model.

In [ ]:
linuncpred = # your code here
regpred = # your code here

Let's plot the results, including the predictions.

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x,y, c = 'red', label = "Data")
plt.xlabel('time (s)')
plt.ylabel('distance (m)')
plt.scatter(X_test, y_test, marker="x", label="Test Data", c="black")
plt.scatter(X_test, y_pred_reg, marker="+", c="green", label="Lin. Reg. Prediction")
plt.scatter(X_test, y_pred_tree, marker="+", c="blue", label="Tree Prediction")
plt.scatter(12, linpred, marker="*", c="m", label="Lin. Fit Prediction")
plt.scatter(12, linuncpred, marker="*", c="c", label="Lin. Fit w Unc Prediction")
plt.scatter(12, treepred, marker="+", c="blue")
plt.scatter(12, regpred, marker="+", c="green")
plt.legend()

### Questions (answer all 4 in the markdown cell below):
1. Which of the two ML models is performing better, in this case? Back up your assessment with at least two pieces of evidence.

2. What are some advantages and disadvantages of a machine learning based-approach, in the context of this problem? Hint: think about the different things we might be trying to achieve with this data - e.g. extracting the velocity and starting position of the cart, predicting future motion (extrapolation), figuring out what the cart was doing between measurements (interpolation). 

3. Did the ML models allow us to take the uncertainty of the measurements into account?

4. We'll see in future weeks that the linear regression model we used should be equivalent to a linear fit. In this case, though, it gave us a different prediction for the point at t = 12 than either of our linear fits. Give 1 reason why that might be the case.


### Acknowledgement statement: every assignment you submit will include an acknowledgement statement crediting the resources (human or otherwise) that you relied on for your work. In this case, your group mates are all already credited, but if you used any other resources, credit them here.

We gratefully acknowledge... (fill in if needed!)

### That's it for this studio! Upload your completed notebook to Gradescope to submit it.